# Dataset 만들기

## 1. 멜론_한국대중음악 수집

### 1) 모듈 불러오기

In [23]:
# 1. 크롬 드라이버 셀레니움 환경 설정하기
## 크롬드라이버 설치 -> 크롬 정보 확인하여 버전에 맞는걸로 Download
### https://googlechromelabs.github.io/chrome-for-testing/#stable

# 2. 셀레니움 설치 완료 후 필요한 모듈 불러오기
from selenium import webdriver                                          # webdriver : 크롬 가상 드라이버 실행
from selenium.webdriver.common.by import By                             # By : 실제 페이지 내용을 긁어올 때 수단을 설정하는 모듈
from selenium.webdriver.common.keys import Keys                         # Keys : 대신 키보드를 눌러주는 동작을 전달
from selenium.webdriver.support.ui import WebDriverWait                 # WebDriverWait : 대기가 필요한 상황에서 사용하기
from selenium.webdriver.support import expected_conditions as EC        # expected_conditions : 요소의 상태를 알아볼 때 사용
from selenium.webdriver.chrome.options import Options                   # Options : 크롬 드라이버의 옵션을 정함. ex) 크롬창 크기 등
from selenium.webdriver.chrome.service import Service                   # Service : Selenium에서 ChromeDriver의 경로를 설정하고 관리하는 역할
from selenium.webdriver.common.action_chains import ActionChains        # ActionCahins : Selenium에서 입력 장치를 사용하여 복잡한 사용자 동작을 자동화
from selenium.common.exceptions import NoSuchElementException           # NoSuchElementException : 지정한 조건에 맞는 요소를 찾을 수 없을 때 발생하는 예외 함수

# 3. BeautifulSoup 활용 할 모듈 불러오기
# from bs4 import BeautifulSoup                       
import time                                     # 시간 관련 작업을 간편하게 처리하기 위해 사용
# import math                                     # 수학적 계산을 위한 다양한 함수들을 제공
import pandas as pd 
# import pickle                                   # 데이터를 저장할 때 사용

### 2) Selenium으로 음원 chart 자료 크롤링

In [36]:
# WebDriver 설정 (ChromeDriver 경로 지정)
service = Service(r"G:\내 드라이브\[Edu] wanted_poptenup\chromedriver-win64\chromedriver.exe")  # ChromeDriver 경로
options = webdriver.ChromeOptions()

# 백그라운드 실행
# options.add_argument('--headless')                      # 브라우저를 GUI 없이 실행
# options.add_argument('--disable-gpu')                   # GPU를 비활성화
# options.add_argument('--no-sandbox')                    # Chcome의 샌드박스 보안 기능을 비활성화 (일부 제한적인 환경에서 안정적으로 실행되도록 설정)

# 브라우저 열기
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

# 멜론뮤직 홈페이지로 이동
url = "https://www.melon.com/"
driver.maximize_window()
# maximize_window()	창을 최대화합니다.
# minimize_window()	창을 최소화합니다.
# fullscreen_window()	창을 전체 화면 모드로 설정합니다.
# set_window_size(w, h)	창의 크기를 설정합니다.
# get_window_size()	창의 크기를 가져옵니다.
# set_window_position(x, y)	창의 위치를 설정합니다.
# get_window_position()	창의 위치를 가져옵니다.
# switch_to.window()	특정 창으로 전환합니다.
driver.get(url)                                             # 멜론 뮤직 차트로 진입
time.sleep(2)                                               # 페이지 로드 대기

# 멜론뮤직 '장르음악' 선택
melon_ganre = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="gnb_menu"]/ul[1]/li[3]/a/span[2]')))
melon_ganre.click()

# 장르 선택하여 클릭
# second_box = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,'')))
# second_box.click()

# '인기순' 클릭
popular_box = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="frm"]/div/div/div/a[2]')))
popular_box.click()

### 3) 곡정보 담기

In [ ]:
# 멜론 차트 list 만들기
title = []
artist = []
lyrics = []
likes = []

# 기본 설정 값
table_all = driver.find_element(By.XPATH, '//*[@id="songList"]')
list_all = driver.find_elements(By.CSS_SELECTOR, "tbody>tr")

# 수집 루틴
for idx in range(0, len(list_all)):
    time.sleep(1)
    try :
        # 스크롤 동적으로 이동 : 인덱스 기반 스크롤
        driver.execute_script(f"window.scrollTo(0, {idx * 85});")

        # 특정 요소가 로드될 때까지 대기
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "tbody>tr")))
        
        # 위치 찾아서 진입
        table_all = driver.find_element(By.XPATH, '//*[@id="songList"]')
        list_all = driver.find_elements(By.CSS_SELECTOR, "tbody>tr")

        # 현재 곡 클릭
        list_click = list_all[idx].find_elements(By.CSS_SELECTOR, "td")
        list_click[3].click()
        song_click = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, f"#frm > div > table > tbody > tr:nth-child({int(idx+1)}) > td:nth-child(4) > div > a:not(.section_hitsong)")))
        song_click.click()

        # 제목 추가
        title_text = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".song_name"))).text
        title.append(title_text)

        # 아티스트 이름 추가
        artist_name = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".artist_name")))
        # artist_row = artist_info.find_element(By.CSS_SELECTOR, "tbody>tr>td")
        # artist_name = artist_row.find_element(By.CSS_SELECTOR, "a").text
        artist.append(artist_name)

        # 가사 텍스트 추가
        lyrics_text = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".lyric"))).text
        lyrics.append(lyrics_text)

        # 좋아요 개수 추가
        likes_text = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".cnt"))).text
        likes.append(likes_text)
        
        # 중간 확인
        if list(range(1, 5000+1, 100) in len(title) :
                print(len(title))
        else :
            continue

        # 뒤로가기
        driver.back()
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="songList"]')))
    except :
        print(f"오류 발생", {idx})        
        break

### 4) 수집량 확인

In [39]:
print(len(title))
print(len(artist))
print(len(lyrics))
print(len(likes))

50
50
50
50


## 2. 파일 내보내기

### 1) 수집 데이터 프레임화

In [40]:
# 데이터 취합하기
data = pd.DataFrame(
    {
        "제목" : title,
        "가수" : artist,
        "가사" : lyrics,
        "좋아요" : likes
    }
)

In [41]:
# 데이터 확인하기
data.head(10)

,제목,가수,가사,좋아요
0,나는 반딧불,<selenium.webdriver.remote.webelement.WebEleme...,나는 내가 빛나는 별인 줄 알았어요\n한 번도 의심한 적 없었죠\n몰랐어요 난 내가...,"84,120"
1,내게 사랑이 뭐냐고 물어본다면,<selenium.webdriver.remote.webelement.WebEleme...,뜨겁게 사랑했던\n계절을 지나\n처음과는 조금은 달라진\n우리 모습을\n걱정 하진 ...,"49,190"
2,소나기,<selenium.webdriver.remote.webelement.WebEleme...,그치지 않기를 바랬죠\n처음 그대 내게로 오던 그날에\n잠시 동안 적시는\n그런 비...,"169,310"
3,천상연,<selenium.webdriver.remote.webelement.WebEleme...,아니길 바랬었어\n꿈이길 기도했지\n너 없는 가슴으로 살아가야 하는 건\n내게는 너...,"114,604"
4,슬픈 초대장,<selenium.webdriver.remote.webelement.WebEleme...,내 야윈 손위로 온 초대장 위에\n널 데려간다는 그와 네 이름\n오래전 헤어지던 날...,"60,911"
5,"어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",<selenium.webdriver.remote.webelement.WebEleme...,일부러 몇 발자국 물러나\n내가 없이 혼자 걷는 널 바라본다\n옆자리 허전한 너의 ...,"477,078"
6,Love wins all,<selenium.webdriver.remote.webelement.WebEleme...,"Dearest, Darling, My universe\n날 데려가 줄래?\n나의 이...","200,410"
7,첫 눈,<selenium.webdriver.remote.webelement.WebEleme...,첫눈 오는 이런 오후에\n너에게 전화를 걸 수만\n있다면 기쁠텐데\n벌써 일년이 지...,"307,734"
8,너의 모든 순간,<selenium.webdriver.remote.webelement.WebEleme...,이윽고 내가 한눈에\n너를 알아봤을 때\n모든 건 분명 달라지고 있었어\n내 세상은...,"304,369"
9,그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection)),<selenium.webdriver.remote.webelement.WebEleme...,날 사랑해서 떠난다며\n눈물짓던 그대의 말을 믿을 수 없죠\n하지만 나의 전부였던\...,"138,158"


### 2) 데이터 내보내기

In [43]:
# 데이터 csv파일로 내보내기
data.to_csv(r"G:\내 드라이브\[Edu] wanted_poptenup\01_프로젝트 모음\25.01_프로젝트#2_'BE MY MUSE' 작사 공모전\(Data)\melon_한국대중음악_발라드(TOP50).csv", index=True, encoding='utf-8')